# Data Cleansing for Spotify Dataset

In [1]:
import os
os.environ["SPARK_HOME"] = "/home/osboxes/spark/spark-2.4.0-bin-hadoop2.7/"

import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import levenshtein 
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#conf = pyspark.SparkConf().setAppName('appName').setMaster('spark://192.168.11.128:8080')

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
df_weeklyTop200 = spark.read.csv('../datasets/spotify_top_200_weekly.csv', header=True)
df_track_details = spark.read.csv('../datasets/spotify_track_details.csv', header=True)
df_kaggle_artists = spark.read.csv('../datasets/kaggle_artists.csv', header=True)
df_musicbrainz_artist = spark.read.csv('../datasets/musicBrainz_ActiveArtists.csv', header=True)

In [3]:
df_artists.count()

NameError: name 'df_artists' is not defined

In [ ]:
df_weeklyTop200.show()

In [ ]:
df_weeklyTop200.info()

In [ ]:
#df_weeklyTop200['Start_Date'] = df_weeklyTop200.apply(lambda row: row.date[:10],row.date[10:] axis = 1) 
df_weeklyTop200.describe(include='all')
#df_weeklyTop200.describe()



In [ ]:
df_weeklyTop200['date']

In [ ]:
#import pyspark.sql.functions as F

In [22]:
from pyspark.sql.functions import col, split
df_weeklyTop200 = df_weeklyTop200.withColumn("start_date", split(col("date"), "--").getItem(0)).withColumn("end_date", split(col("date"), "--").getItem(1))

df_weeklyTop200.show(2)
#df_weeklyTop200[['start_date', 'end_date']] = df_weeklyTop200.date #.str.split('--', expand=True)
#df_weeklyTop200

+--------+--------------------+-------------+-------+--------------------+------+--------------------+----------+----------+
|Position|          Track Name|       Artist|Streams|                date|region|          spotify_id|start_date|  end_date|
+--------+--------------------+-------------+-------+--------------------+------+--------------------+----------+----------+
|       1|   Someone You Loved|Lewis Capaldi|3338234|2019-03-01--2019-...|    gb|2TIlqbIneP0ZY1O0E...|2019-03-01|2019-03-08|
|       2|break up with you...|Ariana Grande|3041558|2019-03-01--2019-...|    gb|4kV4N9D1iKVxx1KLv...|2019-03-01|2019-03-08|
+--------+--------------------+-------------+-------+--------------------+------+--------------------+----------+----------+
only showing top 2 rows



In [ ]:
#df_weeklyTop200.distinct().show()
df_weeklyTop200.select('Artist').distinct().count()

In [ ]:
df_weeklyTop200.select('*').describe()

## Clean up the track_details data
### Lookup missing mb_id in the in the kaggle dataset

In [ ]:
df_track_details.show(2)

In [5]:
#get all rows with no mb_id

df_null_mbid = df_track_details.filter(df_track_details.artist_mbid.isNull())
df_null_mbid.count()


629

In [ ]:
#check if any row has no artist name
df_null_mbid.filter(df_null_mbid.artist_name.isNull()).count()


In [ ]:
#get unique rows with artist_name to avoid expensive join
df_unique_artist = df_null_mbid.dropDuplicates(['artist_name'])
print(df_unique_artist.count())
df_unique_artist.show()

In [ ]:
newDF=df_unique_artist.join(df_artists,levenshtein(df_unique_artist['artist_name'], df_artists['artist_mb']) < 1, 'leftouter')

In [ ]:
newDF.count()

In [ ]:
#print(newDF.count())
newDF[['artist_name','mbid', 'artist_mb']].show()

In [ ]:
#from fuzzywuzzy import fuzz

In [ ]:
#fuzz.token_sort_ratio(Str1,Str2)
#newDF=df_unique_artist.join(df_artists,fuzz.token_sort_ratio(df_unique_artist['artist_name'], df_artists['artist_mb']) > 90, 'left_semi')

In [ ]:
df_musicbrainz_artist.show(2)

In [ ]:
#get unique rows with artist_name to avoid expensive join
df_unique_musicbrainz_artist = df_musicbrainz_artist.dropDuplicates(['sort_name'])
df_unique_musicbrainz_artist.count()

In [ ]:
#get unique rows with df_kaggle_artists to avoid expensive join
df_kaggle_unique_artists = df_kaggle_artists.dropDuplicates(['artist_mb'])
df_kaggle_unique_artists.count()


In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1*1024*1024*1024)
df_unique_artist = df_unique_artist.repartition(1)
df_unique_musicbrainz_artist = df_unique_musicbrainz_artist.repartition(1)
newDF=df_unique_artist.join(df_unique_musicbrainz_artist,levenshtein(df_unique_artist['artist_name'], df_unique_musicbrainz_artist['sort_name']) < 1, 'left')

In [ ]:
newDF.createOrReplaceTempView("MBID_JOIN")

#sql("SELECT col1, col2 FROM V_JOIN")

In [ ]:
newDF.filter(newDF.gid.isNull()).count()

In [ ]:
newDF[['artist_name','gid', 'sort_name']].show()

## Persist datasets to temporary tables

In [ ]:
df_track_details.registerTempTable("track_details")
df_weeklyTop200.registerTempTable("weeklyTop200")

df_kaggle_unique_artists.registerTempTable("kaggle_unique_artists")
df_unique_musicbrainz_artist.registerTempTable("unique_musicbrainz_artist")
df_unique_artist.registerTempTable("unique_artist")

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

rs_artist = sqlContext.sql('''
select ua.track_spotifyID,ua.artist_spotifyID, ua.artist_name,ka.artist_mb, ka.mbid \
from unique_artist ua, kaggle_unique_artists ka \
where ka.artist_mb = ua.artist_name''')
rs_artist.count()


In [ ]:
rs_artist.show()

In [6]:
import requests
import json, csv
import pandas as pd
from pandas.io.json import json_normalize
#from pandas import json_normalize
from datetime import datetime
import numpy as np
API_KEY = 'a28a5c24b56cd7957413ffffe0b4b3fa'

def lastFM_get_mbid(artist_name):
    try:
        r = requests.get('http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist='+ artist_name  +'&api_key='+ API_KEY + '&format=json')
        result = r.content
        json_data = json.loads(r.text)
        df = json_normalize(json_data['artist'])     
        return str(df.mbid[0])
    except:
        return ''

In [7]:
#df_loop = df_null_mbid
ls_mbid = []
for row in df_null_mbid.collect():    
    ls_mbid.append(lastFM_get_mbid(row.artist_name))

In [13]:
df_null_mbid = df_null_mbid.withColumn('mbid', ls_mbid)
df_null_mbid.show()

AssertionError: col should be Column

In [20]:
df_loop  = df_null_mbid
#df_loop = df_null_mbid
ls_mbid = []
for row in df_loop.collect():
    ls_mbid.append(lastFM_get_mbid(row.artist_name))
df_loop['lastFM_mbid'] = ls_mbid

TypeError: 'DataFrame' object does not support item assignment

In [25]:
#df_null_mbid = df_null_mbid.withColumn("mbid2", ls_mbid)

df_loop = df_loop.select("*").toPandas()

In [28]:
df_loop['artist_mbid'] = ls_mbid
df_loop.head()


,track_spotifyID,artist_spotifyID,artist_mbid,artist_name,track_url,track_popularity,track_duration_ms,track_is_local,albumID,album_track_number,album_release_date,album_type
0,5itOtNx0WxtJmi1TQ3RuRd,4f9iBmdUOhQWeP7dcAn1pf,,Rag'n'Bone Man,https://open.spotify.com/track/5itOtNx0WxtJmi1...,80,229184,False,4PwXTHenZZx7ebgsnTM65K,1,2019-01-11,single
1,2VxeLyX666F8uXCJ0dZF8B,4VIvfOurcf0vuLRxLkGnIG,,Bradley Cooper,https://open.spotify.com/track/2VxeLyX666F8uXC...,86,215733,False,4sLtOBOzn4s3GDUv3c5oJD,12,2018-10-05,album
2,2cytBOLpwFRX7J9URCrFIe,31Ua7zSTJxegjyd49ujbSA,ecebea6a-c064-4409-9699-b0bf6b5df720,NSG,https://open.spotify.com/track/2cytBOLpwFRX7J9...,69,240081,False,35MvbFXqp1ESDkKNqYfOIM,1,2018-11-30,single
3,2cytBOLpwFRX7J9URCrFIe,7b79bQFziJFedJb75k6hFt,,Tion Wayne,https://open.spotify.com/track/2cytBOLpwFRX7J9...,69,240081,False,35MvbFXqp1ESDkKNqYfOIM,1,2018-11-30,single
4,1fp2uoWXPca3mIdxRHIgQm,1Gm0kKaDx4GH6pQF88rjSK,c872f796-1497-49d2-8c29-852baa3ede69,Cadet,https://open.spotify.com/track/1fp2uoWXPca3mId...,65,192571,False,4wOuGy0YTmOHjHIUo6drO8,1,2018-09-14,single


In [46]:
df_loop[df_loop['artist_mbid'].isnull()].count()


track_spotifyID       0
artist_spotifyID      0
artist_mbid           0
artist_name           0
track_url             0
track_popularity      0
track_duration_ms     0
track_is_local        0
albumID               0
album_track_number    0
album_release_date    0
album_type            0
dtype: int64